In [1]:
from __future__ import division
from __future__ import print_function

import time
import os, glob, sys
import numpy as np
import itertools
import pandas as pd
from sklearn import svm
from sklearn import metrics


# Get the current working directory
cwd = os.getcwd()
mydir = os.path.abspath(os.path.join(cwd, ".."))
sys.path.append(mydir)
from DataGenerator import genTrainValidFolds

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# SVM Model
-------

In [2]:
# Linear SVC with rbf and different gamma values
class SVM_model():
    def __init__(self, kernel, c_arr, gamma_arr):
        self.c_arr = c_arr#[0.1, 1.0, 10.0]#, 1000.0]            
        self.gamma_arr = gamma_arr#[0.1, 1, 10.0]
        self.kernel=kernel

    def classify(self, trainData, trainLabels, validData):
        pred_dict = {}
        for (c,gamma) in itertools.product(self.c_arr, self.gamma_arr):
            string = "c" + str(c) + "_" + "gamma" + str(gamma)  
#             print ('Running For: ', string)
            clf = svm.SVC(kernel=self.kernel, C=c, gamma=gamma)
            classifier = clf.fit(trainData,trainLabels)
            pred_dict[string] = clf.predict(validData)
        return pred_dict

In [3]:
# Provide the directory of the batches so that we can generate the sequence one after one.

def buildModel(c_arr, gamma_arr, FeatureDir):
    objSVM = SVM_model(kernel='rbf', c_arr=c_arr, gamma_arr=gamma_arr)
    estimatEach_c_gamma_Dict = {}
    validLabelDict = {}
    
    for foldNUM, (trainData, trainLabels, validData, validLabels, labelDict) in enumerate(genTrainValidFolds(FeatureDir, oneHot=False)):
        if foldNUM==0:
            print ('Validation Data and Labels shape: ', validData.shape, validLabels.shape)
            print ('Training Data and Labels shape: ', trainData.shape, trainLabels.shape)
            print ('The Label Dictionary is given as: ', labelDict)
            
        print ('Cross-Vaidation Fold: ', foldNUM)
        
        estimatEach_c_gamma_Dict[foldNUM] = objSVM.classify(trainData=trainData, 
                                                        trainLabels=trainLabels, 
                                                        validData=validData)
        
        validLabelDict[foldNUM] = validLabels
#         break
    return estimatEach_c_gamma_Dict, validLabelDict 

# Performance Metric
-------

In [4]:

def performanceMetric(c_arr, gamma_arr, estimatEach_c_gamma_Dict, validLabelDict):
    performanceDF = pd.DataFrame(np.NaN, index=["c" + str(c) + "_" + "gamma" + str(gamma) for c, gamma in itertools.product(c_arr, gamma_arr)],
                                            columns=['avg-numAirplaneClssified','avg-numCatClassified', 'avg-airplaneAccuracy', 'avg-catAccuracy','avg-totAccuracy'])
#     print (performanceDF)
    for foldNUM, c_gamma_prediction in  estimatEach_c_gamma_Dict.items():
#         print ('Running for cross validation fold : ', foldNUM)
        for c_gamma, estimates in c_gamma_prediction.items():
            confusionMatrix = pd.crosstab(validLabelDict[foldNUM], estimates)
            total = (sum(sum(np.array(confusionMatrix))))
            
            # When our predictions are all 1's 
            # then the column 0 will not exist in the confusion matrix 
            # We handle it here
            if 0 not in confusionMatrix.columns:
                confusionMatrix[0] = np.array([0,0])
                
            # When our predictions are all 0's 
            # then the column 1 will not exist in the confusion matrix 
            # We handle it here
            if 1 not in confusionMatrix.columns:
                confusionMatrix[1] = np.array([0,0])
#            
            if pd.isnull(performanceDF.ix[c_gamma, 'avg-numAirplaneClssified']):
                performanceDF.ix[c_gamma, 'avg-numAirplaneClssified'] = confusionMatrix.ix[0,0]
            else:
                performanceDF.ix[c_gamma, 'avg-numAirplaneClssified'] = (performanceDF.ix[c_gamma, 'avg-numAirplaneClssified'] + 
                                                                      confusionMatrix.ix[0,0])
            if pd.isnull(performanceDF.ix[c_gamma, 'avg-numCatClassified']):
                performanceDF.ix[c_gamma, 'avg-numCatClassified'] = confusionMatrix.ix[1,1]
            else:
                performanceDF.ix[c_gamma, 'avg-numCatClassified'] = (performanceDF.ix[c_gamma, 'avg-numCatClassified'] + 
                                                                      confusionMatrix.ix[1,1])

            if pd.isnull(performanceDF.ix[c_gamma, 'avg-airplaneAccuracy']):
                performanceDF.ix[c_gamma, 'avg-airplaneAccuracy'] = confusionMatrix.ix[0,0]/(confusionMatrix.ix[0,0] + confusionMatrix.ix[0,1]) 
            else:
                performanceDF.ix[c_gamma, 'avg-airplaneAccuracy'] = (performanceDF.ix[c_gamma, 'avg-airplaneAccuracy'] + 
                                                                      confusionMatrix.ix[0,0]/(confusionMatrix.ix[0,0] + confusionMatrix.ix[0,1]))
            if pd.isnull(performanceDF.ix[c_gamma, 'avg-catAccuracy']):
                performanceDF.ix[c_gamma, 'avg-catAccuracy'] = confusionMatrix.ix[1,1]/(confusionMatrix.ix[1,0] + confusionMatrix.ix[1,1]) 
            else:
                performanceDF.ix[c_gamma, 'avg-catAccuracy'] = (performanceDF.ix[c_gamma, 'avg-catAccuracy'] + 
                                                                      confusionMatrix.ix[1,1]/(confusionMatrix.ix[1,0] + confusionMatrix.ix[1,1]))

            if pd.isnull(performanceDF.ix[c_gamma, 'avg-totAccuracy']):
                performanceDF.ix[c_gamma, 'avg-totAccuracy'] = (confusionMatrix.ix[0,0] + confusionMatrix.ix[1,1])/total
            else:
                performanceDF.ix[c_gamma, 'avg-totAccuracy'] = (performanceDF.ix[c_gamma, 'avg-totAccuracy'] + 
                                                                      ((confusionMatrix.ix[0,0] + confusionMatrix.ix[1,1])/total))

    return performanceDF/(foldNUM+1)

# performanceDF



          

# Evaluate Models:
------

In [5]:
HOGp1batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/featureModels/HOGp1/batchData/"
HOGp2batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/featureModels/HOGp2/batchData/"
HOGp3batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/featureModels/HOGp3/batchData/"
HOGp4batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/featureModels/HOGp4/batchData/"

## HOG | Orientation=18 | kernel: (9,9) | c_arr = [0.1, 1.0, 10.0] |  gamma_arr = [0.1, 1.0, 10.0]
-----

In [6]:
c_arr = [0.1, 1.0, 10.0]#, 1000.0]            
gamma_arr = [0.1, 1, 10.0]

FeatureDir = HOGp1batch_dir

start_time = time.time()

estimatEach_c_gamma_Dict, validLabelDict = buildModel(c_arr=c_arr, gamma_arr=gamma_arr, FeatureDir=FeatureDir)

print("--- %s seconds " % (time.time() - start_time) + 
      '--- %s minutes' %str((time.time() - start_time)/60) + 
      '--- %s hours' %str((time.time() - start_time)/3600))

Validation Data and Labels shape:  (1000, 162) (1000,)
Training Data and Labels shape:  (9000, 162) (9000,)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Validation Data and Labels shape:  (1000, 162) (1000,)
Training Data and Labels shape:  (9000, 162) (9000,)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Validation Data and Labels shape:  (1000, 162) (1000,)
Training Data and Labels shape:  (9000, 162) (9000,)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Validation Data and Labels shape:  (1000, 162) (1000,)
Training Data and Labels shape:  (9000, 162) (9000,)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Validation Data and Labels shape:  (1000, 162) (1000,)
Training Data and Labels shape:  (9000, 162) (9000,)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pick

In [12]:
performanceDF = performanceMetric(c_arr=c_arr, 
                                  gamma_arr=gamma_arr,
                                  estimatEach_c_gamma_Dict=estimatEach_c_gamma_Dict, 
                                  validLabelDict=validLabelDict)
performanceDF

,avg-numAirplaneClssified,avg-numCatClassified,avg-airplaneAccuracy,avg-catAccuracy,avg-totAccuracy
c0.1_gamma0.1,396.7,445.1,0.7934,0.8902,0.8418
c0.1_gamma1,430.0,434.9,0.8600,0.8698,0.8649
c0.1_gamma10.0,499.5,27.1,0.9990,0.0542,0.5266
c1.0_gamma0.1,419.4,439.6,0.8388,0.8792,0.8590
c1.0_gamma1,433.6,444.6,0.8672,0.8892,0.8782
c1.0_gamma10.0,472.9,252.6,0.9458,0.5052,0.7255
c10.0_gamma0.1,427.2,442.4,0.8544,0.8848,0.8696
c10.0_gamma1,434.2,440.8,0.8684,0.8816,0.8750
c10.0_gamma10.0,470.5,272.8,0.9410,0.5456,0.7433


## HOG | Orientation=18 | kernel: (6,6) | c_arr = [0.1, 1.0, 10.0] |  gamma_arr = [0.1, 1.0, 10.0]
-----

In [6]:
c_arr = [0.1, 1.0, 10.0]#, 1000.0]            
gamma_arr = [0.1, 1, 10.0]

FeatureDir = HOGp1batch_dir
print (FeatureDir)
start_time = time.time()

estimatEach_c_gamma_Dict, validLabelDict = buildModel(c_arr=c_arr, gamma_arr=gamma_arr, FeatureDir=FeatureDir)

print("--- %s seconds " % (time.time() - start_time) + 
      '--- %s minutes' %str((time.time() - start_time)/60) + 
      '--- %s hours' %str((time.time() - start_time)/3600))

/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/featureModels/HOGp1/batchData/
Validation Data and Labels shape:  (1000, 450) (1000,)
Training Data and Labels shape:  (9000, 450) (9000,)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Cross-Vaidation Fold:  0
Cross-Vaidation Fold:  1
Cross-Vaidation Fold:  2
Cross-Vaidation Fold:  3
Cross-Vaidation Fold:  4
Cross-Vaidation Fold:  5
Cross-Vaidation Fold:  6
Cross-Vaidation Fold:  7
Cross-Vaidation Fold:  8
Cross-Vaidation Fold:  9
--- 3150.894644021988 seconds --- 52.51491085290909 minutes--- 0.8752485155396992 hours


In [37]:
performanceDF = performanceMetric(c_arr=c_arr, 
                                  gamma_arr=gamma_arr,
                                  estimatEach_c_gamma_Dict=estimatEach_c_gamma_Dict, 
                                  validLabelDict=validLabelDict)
performanceDF

,avg-numAirplaneClssified,avg-numCatClassified,avg-airplaneAccuracy,avg-catAccuracy,avg-totAccuracy
c0.1_gamma0.1,423.2,445.4,0.8464,0.8908,0.8686
c0.1_gamma1,455.2,343.1,0.9104,0.6862,0.7983
c0.1_gamma10.0,400.0,100.0,0.8000,0.2000,0.5000
c1.0_gamma0.1,436.4,444.0,0.8728,0.8880,0.8804
c1.0_gamma1,456.3,416.5,0.9126,0.8330,0.8728
c1.0_gamma10.0,400.9,103.9,0.8018,0.2078,0.5048
c10.0_gamma0.1,439.7,447.0,0.8794,0.8940,0.8867
c10.0_gamma1,458.6,419.7,0.9172,0.8394,0.8783
c10.0_gamma10.0,400.9,104.1,0.8018,0.2082,0.5050


## HOG | Orientation=18 | kernel: (4,4) | c_arr = [0.1, 1.0, 10.0] |  gamma_arr = [0.1, 1.0, 10.0]
-----

In [6]:
c_arr = [0.1, 1.0, 10.0]#, 1000.0]            
gamma_arr = [0.1, 1, 10.0]

FeatureDir = HOGp2batch_dir
print (FeatureDir)
start_time = time.time()

estimatEach_c_gamma_Dict, validLabelDict = buildModel(c_arr=c_arr, gamma_arr=gamma_arr, FeatureDir=FeatureDir)

print("--- %s seconds " % (time.time() - start_time) + 
      '--- %s minutes' %str((time.time() - start_time)/60) + 
      '--- %s hours' %str((time.time() - start_time)/3600))

/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/featureModels/HOGp2/batchData/
Validation Data and Labels shape:  (1000, 576) (1000,)
Training Data and Labels shape:  (9000, 576) (9000,)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Cross-Vaidation Fold:  0
Cross-Vaidation Fold:  1
Cross-Vaidation Fold:  2
Cross-Vaidation Fold:  3
Cross-Vaidation Fold:  4
Cross-Vaidation Fold:  5
Cross-Vaidation Fold:  6
Cross-Vaidation Fold:  7
Cross-Vaidation Fold:  8
Cross-Vaidation Fold:  9
--- 4260.221014022827 seconds --- 71.00368368228277 minutes--- 1.1833947291639115 hours


In [7]:
performanceDF = performanceMetric(c_arr=c_arr, 
                                  gamma_arr=gamma_arr,
                                  estimatEach_c_gamma_Dict=estimatEach_c_gamma_Dict, 
                                  validLabelDict=validLabelDict)
performanceDF

,avg-numAirplaneClssified,avg-numCatClassified,avg-airplaneAccuracy,avg-catAccuracy,avg-totAccuracy
c0.1_gamma0.1,427.7,437.3,0.8554,0.8746,0.8650
c0.1_gamma1,100.7,400.0,0.2014,0.8000,0.5007
c0.1_gamma10.0,100.0,400.0,0.2000,0.8000,0.5000
c1.0_gamma0.1,440.6,445.6,0.8812,0.8912,0.8862
c1.0_gamma1,171.4,442.6,0.3428,0.8852,0.6140
c1.0_gamma10.0,100.0,400.0,0.2000,0.8000,0.5000
c10.0_gamma0.1,444.2,445.4,0.8884,0.8908,0.8896
c10.0_gamma1,189.5,433.3,0.3790,0.8666,0.6228
c10.0_gamma10.0,100.0,400.0,0.2000,0.8000,0.5000
